# オートグルーン時系列 - 予測クイックスタート

[Colabで開く](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/stable/docs/tutorials/timeseries/forecasting-quick-start.ipynb) [![SageMaker Studio Labで開く](https://studiolab.sagemaker.aws/studiolab.svg)]...まず、必要なモジュールをインポートします。

In [1]:
!pip install autogluon.timeseries


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.2/116.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.6/285.6 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
# pandasというライブラリをpdという名前でインポートします。
# pandasはデータフレームという表形式のデータを扱うための便利なツールです。
import pandas as pd

# autogluon.timeseriesというライブラリからTimeSeriesDataFrameとTimeSeriesPredictorというクラスをインポートします。
# autogluon.timeseriesは時系列データの予測を自動化するためのライブラリです。
# TimeSeriesDataFrameは時系列データを表すクラスで、pandasのデータフレームに似ていますが、時系列に関する情報を持っています。
# TimeSeriesPredictorは時系列データの予測モデルを作成するためのクラスで、複数のアルゴリズムを自動的に選択して学習し、予測結果を出力します。
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor


/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


autogluon.timeseries`を使用するには、以下の2つのクラスのみが必要である：

- TimeSeriesDataFrame` は複数の時系列からなるデータセットを格納する。- TimeSeriesPredictor`は最適な予測モデルのフィッティング、チューニング、選択を行い、新しい予測を生成する。 autogluon.timeseries`を使用するには、以下の2つのクラスのみが必要である。

- TimeSeriesDataFrame` は複数の時系列からなるデータセットを格納する。最良の予測モデルを含む MSSM データのサブセットをロードし、新しい予測を生成する。

M4 の時系列データセットのサブセットを `pandas.

In [3]:
df = pd.read_csv("https://autogluon.s3.amazonaws.com/datasets/timeseries/m4_hourly_subset/train.csv")
df.head()

,item_id,timestamp,target
0,H1,1750-01-01 00:00:00,605.0
1,H1,1750-01-01 01:00:00,586.0
2,H1,1750-01-01 02:00:00,586.0
3,H1,1750-01-01 03:00:00,559.0
4,H1,1750-01-01 04:00:00,511.0


AutoGluon は[long format](https://doc.dataiku.com/dss/latest/time-series/data-formatting.html#long-format)の時系列データを期待します。 データフレームの各行には、時系列の一意な ID(`"item_id"`)を int または str で表した時系列の観測データ(タイムステップ)が含まれます。

- しかし、AutoGluon が使用する `TimeSeriesDataFrame` を構築する際には、列の名前を提供することが重要です。 AutoGluon は、データが期待されるフォーマットと一致しない場合、例外を発生させます。

In [6]:
# データフレームをTimeSeriesDataFrameに変換します。
train_data = TimeSeriesDataFrame.from_data_frame(
    df, # 元のデータフレームです。
    id_column="item_id", # 時系列IDの列名です。
    timestamp_column="timestamp" # タイムスタンプの列名です。
)

# 変換したTimeSeriesDataFrameの先頭5行を表示します。
train_data.head()


target
item_id timestamp                  
H1      1750-01-01 00:00:00   605.0
        1750-01-01 01:00:00   586.0
        1750-01-01 02:00:00   586.0
        1750-01-01 03:00:00   559.0
        1750-01-01 04:00:00   511.0

TimeSeriesDataFrame`に格納された個々の時系列を_item_と呼ぶ。 例えば、itemは需要予測における異なる商品や、金融データセットにおける異なる銘柄に対応するかもしれない。 この設定は時系列の_panel_とも呼ばれる。
これは、[平均絶対誤差(MASE)](https://en.wikipedia.org/wiki/Mean_absolute_scaled_error)に従った...モデルとは*異なる*ことに注意してください。また、予測したいデータは `TimeSeriesDataFrame` の列 `"target"` に格納されます。

このコードは、時系列データの予測モデルを作成する方法を示しています。

- TimeSeriesPredictorというクラスのインスタンスを作成します。
- prediction_lengthという引数には、未来の何タイムステップ分を予測するかを指定します。ここでは48としています。
- pathという引数には、モデルの保存先のパスを指定します。ここでは"autogluon-m4-hourly"としています。
- targetという引数には、予測対象となる変数の列名を指定します。ここでは"target"としています。
- eval_metricという引数には、モデルの評価指標を指定します。ここでは"MASE"としています。MASEとは平均絶対尺度誤差（Mean Absolute Scaled Error）の略で、時系列データの予測精度を測るための指標です。
- fit()メソッドを呼び出して、モデルを学習させます。
- fit()メソッドの引数には、学習用のTimeSeriesDataFrameを渡します。ここではtrain_dataとしています。
- presetsという引数には、モデルの品質や速度のバランスを指定します。ここでは"medium_quality"としています。これは中程度の品質と速度のバランスを意味します。
- time_limitという引数には、モデルの学習にかける時間の上限を秒単位で指定します。ここでは600としています。これは10分間という意味です。

```python
# 時系列データの予測モデルを作成します。
predictor = TimeSeriesPredictor(
    prediction_length=48, # 未来の48タイムステップ分を予測するように設定します。
    path="autogluon-m4-hourly", # モデルの保存先のパスを設定します。
    target="target", # 予測対象となる変数の列名を設定します。
    eval_metric="MASE", # モデルの評価指標を設定します。
)

# モデルを学習させます。
predictor.fit(
    train_data, # 学習用のTimeSeriesDataFrameを渡します。
    presets="medium_quality", # モデルの品質や速度のバランスを設定します。
    time_limit=600, # モデルの学習にかける時間の上限を設定します。
)
```

In [7]:
# 時系列データの予測モデルを作成します。
predictor = TimeSeriesPredictor(
    prediction_length=48, # 未来の48タイムステップ分を予測するように設定します。
    path="autogluon-m4-hourly", # モデルの保存先のパスを設定します。
    target="target", # 予測対象となる変数の列名を設定します。
    eval_metric="MASE", # モデルの評価指標を設定します。
)

# モデルを学習させます。
predictor.fit(
    train_data, # 学習用のTimeSeriesDataFrameを渡します。
    presets="medium_quality", # モデルの品質や速度のバランスを設定します。
    time_limit=600, # モデルの学習にかける時間の上限を設定します。
)


================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: medium_quality
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'MASE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'medium_quality',
 'num_val_windows': 1,
 'prediction_length': 48,
 'random_seed': None,
 'target': 'target',
 'time_limit': 600,
 'verbosity': 2}
Provided training data set with 148060 rows, 200 items (item = single time series). Average time series length is 740.3. Data frequency is 'H'.
AutoGluon will save models to autogluon-m4-hourly/
AutoGluon will gauge predictive performance using evaluation metric: 'MASE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'target'

Starting training. Start time is 2023-06-27 10:53:37
Models that will be 

ここでは、`"medium_quality"`プリセットを使用し、学習時間を10分（600秒）に制限した。 プリセットは、AutoGluonがどのモデルを適合させようとするかを定義する。
medium_quality`プリセットでは、単純なベースライン（`Naive`、`SeasonalNaive`）、統計モデル（`AutoETS`、`Theta`）、`RecursiveTabular`でラップされたツリーベースのモデルLightGBM、...内部検証セットである。 予測は常に、`train_data`の各時系列の終わりから始まる、次の`prediction_length`タイムステップの予測を含む。

このコードは、学習したモデルを使って時系列データの予測を行う方法を示しています。

- predict()メソッドを呼び出して、予測モデルにTimeSeriesDataFrameを渡します。
- predict()メソッドの戻り値は、予測結果を含むTimeSeriesDataFrameです。
- 予測結果のTimeSeriesDataFrameをpredictionsという変数に代入します。
- predictionsの先頭5行を表示するために、head()メソッドを呼び出します。

```python
# 学習したモデルを使って時系列データの予測を行います。
predictions = predictor.predict(train_data) # 予測モデルにTimeSeriesDataFrameを渡して、予測結果を得ます。

# 予測結果のTimeSeriesDataFrameの先頭5行を表示します。
predictions.head()
```

In [8]:
# 学習したモデルを使って時系列データの予測を行います。
predictions = predictor.predict(train_data) # 予測モデルにTimeSeriesDataFrameを渡して、予測結果を得ます。

# 予測結果のTimeSeriesDataFrameの先頭5行を表示します。
predictions.head()


INFO:lightning_fabric.utilities.seed:Global seed set to 123
Model not specified in predict, will default to the model with the best validation score: RecursiveTabular


mean         0.1         0.2         0.3  \
item_id timestamp                                                             
H1      1750-01-30 04:00:00  621.022377  591.515051  601.644310  608.948218   
        1750-01-30 05:00:00  555.173747  513.444087  527.769022  538.098308   
        1750-01-30 06:00:00  512.971912  461.863725  479.408116  492.058856   
        1750-01-30 07:00:00  478.888969  419.874318  440.132836  454.740653   
        1750-01-30 08:00:00  456.222546  390.242160  412.891872  429.223907   

                                    0.4         0.5         0.6         0.7  \
item_id timestamp                                                             
H1      1750-01-30 04:00:00  615.189139  621.022377  626.855615  633.096535   
        1750-01-30 05:00:00  546.924302  555.173747  563.423191  572.249185   
        1750-01-30 06:00:00  502.868447  512.971912  523.075377  533.884968   
        1750-01-30 07:00:00  467.222493  478.888969  490.555446  503.037286   
        1750-01-30 08:00:00  443.179029  456.222546  469.266063  483.221185   

                                    0.8         0.9  
item_id timestamp                                    
H1      1750-01-30 04:00:00  640.400443  650.529702  
        1750-01-30 05:00:00  582.578471  596.903406  
        1750-01-30 06:00:00  546.535708  564.080099  
        1750-01-30 07:00:00  517.645103  537.903620  
        1750-01-30 08:00:00  499.553220  522.202931

AutoGluonは_probabilistic_な予測を行います：時系列の平均（期待値）を将来予測することに加えて、モデルは予測分布の分位数も提供します。 分位数予測は、起こり得る結果の範囲についてのアイデアを与えてくれます。 例えば、`"0.1"`の分位数が`500.0`に等しいとすると、モデルは目標値が`500.0`以下になる確率が10%であると予測していることを意味します。

ここで、データセットの時系列の1つについて、予測値と実際に観測された値を可視化します。 予測値の平均と、潜在的な結果の範囲を示すために10%分位点と90%分位点をプロットします。

In [ ]:
import matplotlib.pyplot as plt

# TimeSeriesDataFrame can also be loaded directly from a file
test_data = TimeSeriesDataFrame.from_path("https://autogluon.s3.amazonaws.com/datasets/timeseries/m4_hourly_subset/test.csv")

plt.figure(figsize=(20, 3))

item_id = "H1"
y_past = train_data.loc[item_id]["target"]
y_pred = predictions.loc[item_id]
y_test = test_data.loc[item_id]["target"][-48:]

plt.plot(y_past[-200:], label="Past time series values")
plt.plot(y_pred["mean"], label="Mean forecast")
plt.plot(y_test, label="Future time series values")

plt.fill_between(
    y_pred.index, y_pred["0.1"], y_pred["0.9"], color="red", alpha=0.1, label=f"10%-90% confidence interval"
)
plt.legend();

## 様々なモデルの性能を評価する

AutoGluonが学習した各モデルの性能は、`leaderboard()`メソッドで見ることができます。 leaderboard関数にテストデータセットを提供し、未知のテストデータに対して我々の適合したモデルがどの程度良いかを確認します。 leaderboardには、内部検証データセットで計算された検証スコアも含まれます。

AutoGluonのリーダーボードでは、より高いスコアは常により良い予測性能に対応します。 したがって、我々のMASEスコアは`-1`倍され、より高い "負のMASE "はより正確な予測に対応します。

In [ ]:
# The test score is computed using the last
# prediction_length=48 timesteps of each time series in test_data
predictor.leaderboard(test_data, silent=True)

## まとめ `autogluon.timeseries` を使って、M4 Hourlyデータセットに対して確率的な多段階予測を行いました。 時系列予測のためのAutoGluonの高度な機能については、[時系列予測 - 詳細](forecasting-indepth.ipynb)をチェックしてください。